# 성적 + 가치관 기반 추천시스템

- 성적정보와 가치관정보를 모두 코사인유사도를 기반으로 분석하여 적절한 팀메이트 추천
- 성적정보 : 총점은 비슷하고, 영역별 점수는 달라야함
- 가치관정보 : 영역별 점수가 최대한 유사해야함

In [2]:
import numpy as np
import pandas as pd

# 점수 0-10점으로 랜덤 생성
# 인원 수 160명
user_grade = pd.DataFrame(np.random.randint(1,11,size=(160,4)) , index=range(1,161), columns=['데이터분석','머신러닝','웹','디자인'])
# 각 유저(사용자)별 네개의 영역 점수 총합
user_grade_sum = user_grade.sum(axis=1)

display(user_grade)
display(user_grade_sum)

,데이터분석,머신러닝,웹,디자인
1,2,1,5,4
2,7,6,3,1
3,8,3,7,1
4,4,4,7,7
5,1,2,6,3
...,...,...,...,...
156,5,6,6,1
157,1,3,2,2
158,5,9,5,8
159,8,8,2,10


1      12
2      17
3      19
4      22
5      12
       ..
156    18
157     8
158    27
159    28
160    21
Length: 160, dtype: int64

In [3]:
# 가치관점수 : 
# 디테일 <-> 속도 : 디테일을 중요시하는지 속도를 중요시하는지
# 과정 <-> 결과 : 과정을 중요시하는지 결과를 중요시하는지
# 창업 <-> 취업 : 창업을 꿈꾸는지 취업을 꿈꾸는지
# 아침 <-> 밤 : 아침형인간인지 밤형 인간인지

# 점수 0-100점으로 랜덤 생성
user_style = pd.DataFrame(np.random.randint(1,101,size=(160,4)) , index=range(1,161), columns=['속도','결과','취업','밤'])
user_style['디테일'] = 100 - user_style['속도']
user_style['과정'] = 100 - user_style['결과']
user_style['창업'] = 100 - user_style['취업']
user_style['아침'] = 100 - user_style['밤']
display(user_style)

,속도,결과,취업,밤,디테일,과정,창업,아침
1,15,24,38,85,85,76,62,15
2,96,86,43,39,4,14,57,61
3,6,91,13,39,94,9,87,61
4,57,90,13,4,43,10,87,96
5,85,90,7,43,15,10,93,57
...,...,...,...,...,...,...,...,...
156,59,73,15,1,41,27,85,99
157,48,22,22,8,52,78,78,92
158,17,63,85,30,83,37,15,70
159,16,97,93,18,84,3,7,82


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
# 성적 기반 유사도 (낮을수록 추천도 상승)
user_based_collab_grade = cosine_similarity(user_grade, user_grade)
print(user_based_collab_grade)

[[1.         0.5899618  0.77107482 ... 0.80244887 0.71632286 0.65559866]
 [0.5899618  1.         0.88808978 ... 0.82288456 0.80830535 0.68015256]
 [0.77107482 0.88808978 1.         ... 0.71026894 0.66301213 0.47382185]
 ...
 [0.80244887 0.82288456 0.71026894 ... 1.         0.94970795 0.95525853]
 [0.71632286 0.80830535 0.66301213 ... 0.94970795 1.         0.91293265]
 [0.65559866 0.68015256 0.47382185 ... 0.95525853 0.91293265 1.        ]]


In [5]:
# 1번 입장에서 진행, 각 점수에서 1번점수를 빼고 절대값을 씌워 1번 사용자의 총합과 얼마나 차이나는지 결정
# 0.1을 곱한 이유? 그대로 사용하면 총합점수차 지표의 영향력이 너무 커진다. 
grade_subs = np.abs(user_grade_sum - user_grade_sum[1]) * 0.1

display(grade_subs)

1      0.0
2      0.5
3      0.7
4      1.0
5      0.0
      ... 
156    0.6
157    0.4
158    1.5
159    1.6
160    0.9
Length: 160, dtype: float64

In [6]:
# 가치관에 대한 유사도
# 높을 수록 추천도가 높아진다.

user_based_collab_style = cosine_similarity(user_style, user_style)
print(user_based_collab_style)

[[1.         0.5328082  0.74392218 ... 0.74164049 0.56760672 0.83913073]
 [0.5328082  1.         0.68638991 ... 0.67471111 0.68542033 0.70781792]
 [0.74392218 0.68638991 1.         ... 0.7820367  0.77791923 0.96603636]
 ...
 [0.74164049 0.67471111 0.7820367  ... 1.         0.95951576 0.90414871]
 [0.56760672 0.68542033 0.77791923 ... 0.95951576 1.         0.8644973 ]
 [0.83913073 0.70781792 0.96603636 ... 0.90414871 0.8644973  1.        ]]


In [7]:
# 높은 추천도를 가지려면?
# user_based_collab(성적영역 유사도)가 낮아야함 (각 영역별 수치가 최대한 달라야 하므로)
# grade_subs(총점수차)가 낮아야함. (차이가 적어야 비슷한 실력이므로)
# user_based_collab_style(가치관 유사도)는 높아야함

# 아래와 같은 수식을 최종 지표로 활용한다.
# 해당 최종 지표가 낮을수록 추천도는 높다
evaluation_value = user_based_collab_grade[0] + grade_subs - user_based_collab_style[0]

evaluation_value

1      1.110223e-16
2      5.571536e-01
3      7.271526e-01
4      1.437170e+00
5      3.863941e-01
           ...     
156    7.777773e-01
157    4.728503e-01
158    1.560808e+00
159    1.748716e+00
160    7.164679e-01
Length: 160, dtype: float64

In [8]:
# 해당 지표가 낮은 순서대로 index 뽑아옴
top10 = evaluation_value.sort_values()[:10].index.tolist()

# 추천되는 인원의 인덱스 리스트
top10

[138, 123, 22, 88, 17, 125, 1, 112, 70, 143]

In [9]:
# 주어진 유저가 어떤 스타일인지(가치관인지) 확인하는 함수

def checkStyle(user):
    style = []
    if user['디테일'] > user['속도']:
        style.append('디테일')
    else:
        style.append('속도')
    if user['과정'] > user['결과']:
        style.append('과정')
    else:
        style.append('결과')
    if user['창업'] > user['취업']:
        style.append('창업')
    else:
        style.append('취업')
    if user['아침'] > user['밤']:
        style.append('아침')
    else:
        style.append('밤')
    return style

In [11]:
# 위와 같은 원리로 인풋 값에 대해 추천시스템 작동
# 가치관이 비슷하고,
# 성적총점은 비슷하지만 잘하는 영역은 다른 유저를 추천해준다.

input_num = int(input()) - 1
grade_subs = np.abs(user_grade_sum - user_grade_sum[input_num + 1]) * 0.1
evaluation_value = user_based_collab_grade[input_num] + grade_subs - user_based_collab_style[input_num]
top10 = evaluation_value.sort_values()[:10].index.tolist()

print(f"<나의 정보>")
print()
print(f"{checkStyle(user_style.iloc[input_num])}")
print(f"데이터분석 : {user_grade.iloc[input_num]['데이터분석']}, 머신러닝 : {user_grade.iloc[input_num]['머신러닝']}, 웹 : {user_grade.iloc[input_num]['웹']}, 디자인 : {user_grade.iloc[input_num]['디자인']}, 총합 : {user_grade_sum[input_num + 1]}")
print(f"강점 : {user_grade.iloc[input_num].sort_values(ascending=False)[:2].index.tolist()}")
print()

print("================================")
for idx, value in enumerate(top10):
    value = value - 1
    print()
    print(f"<추천 {idx + 1}순위 : {value}번째 학생>")
    print()
    print(f"{checkStyle(user_style.iloc[value])}")
    print(f"데이터분석 : {user_grade.iloc[value]['데이터분석']}, 머신러닝 : {user_grade.iloc[value]['머신러닝']}, 웹 : {user_grade.iloc[value]['웹']}, 디자인 : {user_grade.iloc[value]['디자인']}, 총합 : {user_grade_sum[value + 1]}")
    print(f"강점 : {user_grade.iloc[value].sort_values(ascending=False)[:2].index.tolist()}")
    print()
    print("=========================")

<나의 정보>

['속도', '결과', '창업', '아침']
데이터분석 : 1, 머신러닝 : 2, 웹 : 6, 디자인 : 3, 총합 : 12
강점 : ['웹', '디자인']


<추천 1순위 : 142번째 학생>

['속도', '결과', '창업', '아침']
데이터분석 : 5, 머신러닝 : 4, 웹 : 1, 디자인 : 1, 총합 : 11
강점 : ['데이터분석', '머신러닝']


<추천 2순위 : 111번째 학생>

['속도', '결과', '창업', '아침']
데이터분석 : 5, 머신러닝 : 6, 웹 : 1, 디자인 : 2, 총합 : 14
강점 : ['머신러닝', '데이터분석']


<추천 3순위 : 21번째 학생>

['속도', '결과', '취업', '밤']
데이터분석 : 7, 머신러닝 : 1, 웹 : 1, 디자인 : 4, 총합 : 13
강점 : ['데이터분석', '디자인']


<추천 4순위 : 137번째 학생>

['속도', '결과', '취업', '밤']
데이터분석 : 6, 머신러닝 : 6, 웹 : 1, 디자인 : 1, 총합 : 14
강점 : ['머신러닝', '데이터분석']


<추천 5순위 : 122번째 학생>

['속도', '결과', '취업', '밤']
데이터분석 : 2, 머신러닝 : 7, 웹 : 3, 디자인 : 1, 총합 : 13
강점 : ['머신러닝', '웹']


<추천 6순위 : 91번째 학생>

['속도', '결과', '취업', '아침']
데이터분석 : 2, 머신러닝 : 4, 웹 : 1, 디자인 : 6, 총합 : 13
강점 : ['디자인', '머신러닝']


<추천 7순위 : 107번째 학생>

['속도', '과정', '창업', '아침']
데이터분석 : 7, 머신러닝 : 1, 웹 : 2, 디자인 : 4, 총합 : 14
강점 : ['데이터분석', '디자인']


<추천 8순위 : 4번째 학생>

['속도', '결과', '창업', '아침']
데이터분석 : 1, 머신러닝 : 2, 웹 : 6, 디자인 : 3, 총합 : 12
강점 : ['웹', '디